In [ ]:
import h5py as h5
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.models as models
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os
import csv
import h5py
from torchvision.models import resnet18
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from PIL import Image
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import warnings 
warnings.filterwarnings("ignore")

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
train_path = "/kaggle/input/deeplense-task1/dataset/train"
train_imgs = {}
for cat in os.listdir(train_path):
    sub_path = os.path.join(train_path,cat)
    imgs = []
    for img_path in os.listdir(sub_path):
        imgs.append(os.path.join(sub_path,img_path))
    train_imgs[cat] = imgs

In [3]:
val_path = "/kaggle/input/deeplense-task1/dataset/val"
val_imgs = {}
for cat in os.listdir(val_path):
    sub_path = os.path.join(val_path,cat)
    imgs = []
    for img_path in os.listdir(sub_path):
        imgs.append(os.path.join(sub_path,img_path))
    val_imgs[cat] = imgs

In [4]:
train_rows = [(img_path, label) for label, img_paths in train_imgs.items() for img_path in img_paths]
val_rows = [(img_path, label) for label, img_paths in val_imgs.items() for img_path in img_paths]

In [5]:
df_train = pd.DataFrame(train_rows, columns = ['img_path','label'])
df_val = pd.DataFrame(val_rows, columns = ['img_path','label'])

In [6]:
df_train['label'] = df_train['label'].map({'no':0, 'sphere':1, 'vort':2})
df_val['label'] = df_val['label'].map({'no':0, 'sphere':1, 'vort':2})

In [7]:
class Dataset(nn.Module):
    def __init__(self,data,transform = None, train = True):
        self.data = data
        self.imgs = [np.load(img_path) for img_path in self.data['img_path']]
        self.y = torch.tensor(self.data['label'], dtype = torch.long)
        self.transform = transform
        self.train = train

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.imgs[idx].squeeze()[:,:,np.newaxis]
        label = self.y[idx]
        img = np.tile(img,(1,1,3))
        if self.transform != None:
            img = self.transform(image = img)["image"]

        return img, label

In [8]:
train_transform = A.Compose([
    # A.RandomRotate90(p=0.5),
    # A.HorizontalFlip(p=0.5),
    # A.VerticalFlip(p=0.5),
    # A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=45, p=0.5),
    # A.Normalize(mean=(0.485, 0.456, 0.406), 
    #             std=(0.229, 0.224, 0.225)),
    # A.Resize(28,28),
    ToTensorV2()
])

val_transform = A.Compose([
    # A.Resize(28,28),
    ToTensorV2()
])

In [9]:
train_dataset = Dataset(df_train, train_transform)
val_dataset = Dataset(df_val, val_transform)

In [10]:
train_loader = DataLoader(train_dataset, batch_size = 32, shuffle = True, num_workers = 0)
val_loader = DataLoader(val_dataset, batch_size = 32, shuffle = False, num_workers = 0)

In [ ]:
os.makedirs("model_plots", exist_ok=True)

def get_pretrained_encoder(model_name, latent_dim, input_size=(3, 150, 150)):
    if hasattr(models, model_name):
        backbone = getattr(models, model_name)(pretrained=True)
    else:
        raise ValueError(f"Unsupported model: {model_name}")
    if model_name.startswith('efficientnet'):
        encoder = backbone.features
        encoder = nn.Sequential(
            encoder,
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten()
        )
    elif model_name.startswith('densenet'):
        encoder = nn.Sequential(
            backbone.features,
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten()
        )
    else:
        encoder = nn.Sequential(*list(backbone.children())[:-1], nn.Flatten())
    dummy_input = torch.randn(1, *input_size)
    with torch.no_grad():
        dummy_output = encoder(dummy_input)
    feature_dim = dummy_output.shape[1]
    projection = nn.Linear(feature_dim, latent_dim)
    return nn.Sequential(encoder, projection), feature_dim

class AutoencoderClassifier(nn.Module):
    def __init__(self, model_name="resnet18", latent_dim=128, num_classes=3, input_size=(3, 150, 150)):
        super(AutoencoderClassifier, self).__init__()
        self.encoder, feature_dim = get_pretrained_encoder(model_name, latent_dim, input_size)
        self.input_size = input_size
        if model_name.startswith('efficientnet'):
            backbone = getattr(models, model_name)(pretrained=True)
            temp_encoder = backbone.features
        elif model_name.startswith('densenet'):
            backbone = getattr(models, model_name)(pretrained=True)
            temp_encoder = nn.Sequential(backbone.features, nn.ReLU(inplace=True))
        else:
            backbone = getattr(models, model_name)(pretrained=True)
            temp_encoder = nn.Sequential(*list(backbone.children())[:-2])
        dummy_input = torch.randn(1, *input_size)
        with torch.no_grad():
            feature_map = temp_encoder(dummy_input)
        self.feature_map_shape = feature_map.shape[1:]
        num_decoder_features = self.feature_map_shape[0] * self.feature_map_shape[1] * self.feature_map_shape[2]
        self.decoder_fc = nn.Linear(latent_dim, num_decoder_features)
        self.decoder = nn.Sequential(
            nn.Unflatten(1, self.feature_map_shape),
            nn.ConvTranspose2d(self.feature_map_shape[0], 128, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, kernel_size=3, stride=1, padding=1),
            nn.Sigmoid()
        )
        self.classifier = nn.Sequential(
            nn.Linear(latent_dim, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )
    
    def forward(self, x):
        latent = self.encoder(x)
        decoder_input = self.decoder_fc(latent)
        reconstruction = self.decoder(decoder_input)
        class_out = self.classifier(latent)
        if reconstruction.shape[2:] != torch.Size([self.input_size[1], self.input_size[2]]):
            reconstruction = nn.functional.interpolate(
                reconstruction, 
                size=(self.input_size[1], self.input_size[2]), 
                mode='bilinear', 
                align_corners=False
            )
        return class_out, reconstruction, latent

def compute_laplacian(images):
    batch_size, channels, height, width = images.shape

    sobel_x = torch.tensor([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=torch.float32).to(images.device)
    sobel_x = sobel_x.view(1, 1, 3, 3).repeat(channels, 1, 1, 1)
    
    sobel_y = torch.tensor([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=torch.float32).to(images.device)
    sobel_y = sobel_y.view(1, 1, 3, 3).repeat(channels, 1, 1, 1)
    
    padding = nn.ReflectionPad2d(1)
    padded_images = padding(images)
    
    grad_x = F.conv2d(padded_images, sobel_x.expand(channels, 1, 3, 3), groups=channels)
    grad_y = F.conv2d(padded_images, sobel_y.expand(channels, 1, 3, 3), groups=channels)
    
    padded_grad_x = padding(grad_x)
    padded_grad_y = padding(grad_y)
    
    grad_xx = F.conv2d(padded_grad_x, sobel_x.expand(channels, 1, 3, 3), groups=channels)
    grad_yy = F.conv2d(padded_grad_y, sobel_y.expand(channels, 1, 3, 3), groups=channels)

    laplacian = grad_xx + grad_yy
    
    return laplacian

def train_variant(model, train_dataloader, val_dataloader, variant="baseline", num_epochs=10, 
                 device='cuda', lambda_recon=1.0, lambda_pde=0.1, lambda_energy=0.1):
    device = torch.device(device if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    cls_criterion = nn.CrossEntropyLoss()
    recon_criterion = nn.MSELoss()

    history = {
        'train_loss': [], 'train_acc': [], 'train_auc': [],
        'val_loss': [], 'val_acc': [], 'val_auc': [],
        'component_losses': {'cls': [], 'recon': [], 'pde': [], 'energy': []}
    }
    
    all_latents = []
    all_labels = []
    val_labels = []
    
    for epoch in range(num_epochs):
        model.train()
        total_loss, total, correct = 0.0, 0, 0
        all_preds, all_labels_epoch = [], []
        component_losses = {'cls': 0.0, 'recon': 0.0, 'pde': 0.0, 'energy': 0.0}
        
        for images, labels in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs} [Train - {variant}]"):
            images, labels = images.to(device).float(), labels.to(device)
            optimizer.zero_grad()
            outputs, recon, latent = model(images)

            cls_loss = cls_criterion(outputs, labels)
            component_losses['cls'] += cls_loss.item() * images.size(0)
  
            recon_loss = recon_criterion(recon, images)
            component_losses['recon'] += recon_loss.item() * images.size(0)

            loss = cls_loss + lambda_recon * recon_loss

            pde_loss = torch.tensor(0.0).to(device)
            if variant in ["pde", "both"]:
                laplacian = compute_laplacian(recon)
                pde_loss = torch.mean(laplacian**2) 
                loss += lambda_pde * pde_loss
                component_losses['pde'] += pde_loss.item() * images.size(0)
   
            energy_loss = torch.tensor(0.0).to(device)
            if variant in ["energy", "both"]:
                energy_loss = torch.mean((latent**2 - 1)**2) 
                loss += lambda_energy * energy_loss
                component_losses['energy'] += energy_loss.item() * images.size(0)
            
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item() * images.size(0)
            _, preds = outputs.max(1)
            total += labels.size(0)
            correct += preds.eq(labels).sum().item()
            all_preds.extend(outputs.softmax(dim=1).detach().cpu().numpy())
            all_labels_epoch.extend(labels.cpu().numpy())
            
        train_acc = correct / total
        train_auc = roc_auc_score(all_labels_epoch, all_preds, multi_class='ovr')

        for k in component_losses:
            component_losses[k] /= total
            history['component_losses'][k].append(component_losses[k])

        history['train_loss'].append(total_loss / total)
        history['train_acc'].append(train_acc)
        history['train_auc'].append(train_auc)
        
        print(f"Epoch {epoch+1}: Train Loss {total_loss/total:.4f}, Train Acc {train_acc:.4f}, Train AUC {train_auc:.4f}")

        model.eval()
        val_loss, val_correct, total_val = 0.0, 0, 0
        val_preds, val_labels_epoch = [], []
        val_latents = []
        
        
        with torch.no_grad():
            for images, labels in tqdm(val_dataloader, desc=f"Epoch {epoch+1}/{num_epochs} [Val - {variant}]"):
                images, labels = images.to(device).float(), labels.to(device)
                outputs, recon, latent = model(images)

                if epoch == num_epochs - 1:
                    val_latents.append(latent.cpu())
                    val_labels.extend(labels.cpu().numpy())
                
                cls_loss = cls_criterion(outputs, labels)
                recon_loss = recon_criterion(recon, images)

                loss = cls_loss + lambda_recon * recon_loss
                
                if variant in ["pde", "both"]:
                    laplacian = compute_laplacian(recon)
                    pde_loss = torch.mean(laplacian**2)
                    loss += lambda_pde * pde_loss
                
                if variant in ["energy", "both"]:
                    energy_loss = torch.mean((latent**2 - 1)**2)
                    loss += lambda_energy * energy_loss
                
                val_loss += loss.item() * images.size(0)
                _, preds = outputs.max(1)
                total_val += labels.size(0)
                val_correct += preds.eq(labels).sum().item()
                val_preds.extend(outputs.softmax(dim=1).cpu().numpy())
                val_labels_epoch.extend(labels.cpu().numpy())
        
        val_acc = val_correct / total_val
        val_auc = roc_auc_score(val_labels_epoch, val_preds, multi_class='ovr')

        history['val_loss'].append(val_loss / total_val)
        history['val_acc'].append(val_acc)
        history['val_auc'].append(val_auc)
        
        print(f"Epoch {epoch+1}: Val Loss {val_loss/total_val:.4f}, Val Acc {val_acc:.4f}, Val AUC {val_auc:.4f}")

    if val_latents:
        all_latents = torch.cat(val_latents, dim=0)
        all_labels = np.array(val_labels)
    
    return model, history, all_latents, all_labels

def plot_history(histories, variant_names, save_path="model_plots"):
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))

    ax = axes[0, 0]
    for variant, history in zip(variant_names, histories):
        ax.plot(history['train_loss'], label=f"{variant} Train")
        ax.plot(history['val_loss'], label=f"{variant} Val", linestyle='--')
    ax.set_title('Loss')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')
    ax.legend()
    ax.grid(True)
    
    ax = axes[0, 1]
    for variant, history in zip(variant_names, histories):
        ax.plot(history['train_acc'], label=f"{variant} Train")
        ax.plot(history['val_acc'], label=f"{variant} Val", linestyle='--')
    ax.set_title('Accuracy')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Accuracy')
    ax.legend()
    ax.grid(True)

    ax = axes[1, 0]
    for variant, history in zip(variant_names, histories):
        ax.plot(history['train_auc'], label=f"{variant} Train")
        ax.plot(history['val_auc'], label=f"{variant} Val", linestyle='--')
    ax.set_title('ROC AUC')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('AUC')
    ax.legend()
    ax.grid(True)

    ax = axes[1, 1]
    for component in ['cls', 'recon', 'pde', 'energy']:
        if histories[-1]['component_losses'][component]: 
            ax.plot(histories[-1]['component_losses'][component], label=component)
    ax.set_title(f'Component Losses ({variant_names[-1]})')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')
    ax.legend()
    ax.grid(True)
    
    plt.tight_layout()
    plt.savefig(f"{save_path}/learning_curves.png")
    plt.close()
    
    plt.figure(figsize=(14, 8))
    for i, (variant, history) in enumerate(zip(variant_names, histories)):
        plt.subplot(2, 2, i+1)
        for component in ['cls', 'recon', 'pde', 'energy']:
            if history['component_losses'][component]:
                plt.plot(history['component_losses'][component], label=component)
        plt.title(f'Component Losses ({variant})')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(f"{save_path}/component_losses.png")
    plt.close()

def visualize_latent_space(latents, labels, variant, save_path="model_plots"):
    """Visualize the latent space of a model using t-SNE."""
    latents_np = latents.numpy()
    labels_np = np.array(labels)

    tsne = TSNE(n_components=2, random_state=42)
    latents_2d = tsne.fit_transform(latents_np)

    plt.figure(figsize=(8, 6))
    scatter = plt.scatter(latents_2d[:,0], latents_2d[:,1], c=labels_np, cmap='viridis', s=10, alpha=0.8)
    plt.colorbar(scatter, label='Class')
    plt.title(f"t-SNE of Latent Space ({variant})")
    plt.xlabel("Dimension 1")
    plt.ylabel("Dimension 2")
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig(f"{save_path}/latent_space_{variant}.png")
    plt.close()

def main(train_dataloader, val_dataloader, model_name="efficientnet_b0", 
         latent_dim=128, num_classes=3, input_size=(3, 150, 150), num_epochs=10):
    variants = [
        {"name": "baseline", "params": {"variant": "baseline"}},
        {"name": "pde", "params": {"variant": "pde", "lambda_pde": 0.1}},
        {"name": "energy", "params": {"variant": "energy", "lambda_energy": 0.1}},
        {"name": "both", "params": {"variant": "both", "lambda_pde": 0.1, "lambda_energy": 0.1}}
    ]

    all_histories = []
    models = []
    for variant in variants:
        print(f"\n=== Training {variant['name']} variant ===\n")
        model = AutoencoderClassifier(model_name=model_name, latent_dim=latent_dim, 
                                      num_classes=num_classes, input_size=input_size)
        
        model, history, latents, labels = train_variant(
            model, train_dataloader, val_dataloader, 
            num_epochs=num_epochs, **variant["params"]
        )
        models.append(model.state_dict())
        
        all_histories.append(history)

        if latents is not None and len(latents) > 0:
            visualize_latent_space(latents, labels, variant['name'])
    
    variant_names = [v["name"] for v in variants]
    plot_history(all_histories, variant_names)
    
    print("\nTraining and visualization complete. Check the 'model_plots' directory for results.")
    return models

models = main(train_loader, val_loader, num_epochs=5) 


=== Training baseline variant ===



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/5 [Train - baseline]: 100%|██████████| 938/938 [01:19<00:00, 11.76it/s]


Epoch 1: Train Loss 0.9301, Train Acc 0.5249, Train AUC 0.7300


Epoch 1/5 [Val - baseline]: 100%|██████████| 235/235 [00:06<00:00, 35.30it/s]


Epoch 1: Val Loss 0.6713, Val Acc 0.7104, Val AUC 0.8800


...

Epoch 4: Val Loss 0.3075, Val Acc 0.8956, Val AUC 0.9785


Epoch 5/5 [Train - both]: 100%|██████████| 938/938 [01:22<00:00, 11.40it/s]


Epoch 5: Train Loss 0.2679, Train Acc 0.9063, Train AUC 0.9807


Epoch 5/5 [Val - both]: 100%|██████████| 235/235 [00:06<00:00, 34.02it/s]


Epoch 5: Val Loss 0.2950, Val Acc 0.9012, Val AUC 0.9796

Training and visualization complete. Check the 'model_plots' directory for results.


In [30]:
for i,model in enumerate(models):
    torch.save({"model_state_dict":model},f'autoencoder{i}.pth')